In [1]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
def extract_n_samples(data, n, label):
    return data[b'data'][np.where(np.array(data[b'labels'])==label)][:n].reshape(n, 3, 1024).astype(float)

In [4]:
def batch_to_grayscale(img_batch):
    res = []
    for img in np.rollaxis(img_batch, 0):
        tmp = [int((x[0]+x[1]+x[2])/3) for x in np.rollaxis(img, 1)]
        res.append(tmp)
    return np.array(res)

In [5]:
def get_l2(input_hist, targ_hists):
    return np.asarray(list(map(lambda x : np.linalg.norm(input_hist - x), targ_hists)))

In [6]:
# autos 0 / ships 1 / deers 2

def predict_class(hist, autos_hists, ships_hists, deers_hists):
    autos_min_l2 = np.amin(get_l2(hist, autos_hists))
    ships_min_l2 = np.amin(get_l2(hist, ships_hists))
    deers_min_l2 = np.amin(get_l2(hist, deers_hists))
        
    min_index = np.asarray([autos_min_l2, ships_min_l2, deers_min_l2]).argmin()
    
    return min_index

In [7]:
n_samples_train = 30
n_samples_test = 10
bins = 50

# Trainig

In [8]:
train_data = unpickle('cifar-10-python/cifar-10-batches-py/data_batch_1')

In [9]:
autos = extract_n_samples(train_data, n_samples_train, 1)
deers = extract_n_samples(train_data, n_samples_train, 4)
ships = extract_n_samples(train_data, n_samples_train, 8)

In [10]:
train_gs_autos = batch_to_grayscale(autos)
train_gs_deers = batch_to_grayscale(deers)
train_gs_ships = batch_to_grayscale(ships)

In [11]:
train_autos_hists = []
train_deers_hists = []
train_ships_hists = []

for i in range(n_samples_train):
    train_ships_hists.append(np.histogram(train_gs_ships[i], bins=bins)[0])
    train_deers_hists.append(np.histogram(train_gs_deers[i], bins=bins)[0])
    train_autos_hists.append(np.histogram(train_gs_autos[i], bins=bins)[0])
    
np.save('train_ships_hists', np.asarray(train_ships_hists))
np.save('train_deers_hists', np.asarray(train_deers_hists))
np.save('train_autos_hists', np.asarray(train_autos_hists))

# Testing

In [12]:
test_data = unpickle('cifar-10-python/cifar-10-batches-py/test_batch')

In [13]:
autos = extract_n_samples(test_data, n_samples_test, 1)
deers = extract_n_samples(test_data, n_samples_test, 4)
ships = extract_n_samples(test_data, n_samples_test, 8)

In [14]:
test_gs_autos = batch_to_grayscale(autos)
test_gs_deers = batch_to_grayscale(deers)
test_gs_ships = batch_to_grayscale(ships)

In [15]:
test_autos_hists = []
test_deers_hists = []
test_ships_hists = []

for i in range(n_samples_test):
    test_ships_hists.append(np.histogram(test_gs_ships[i], bins=bins)[0])
    test_deers_hists.append(np.histogram(test_gs_deers[i], bins=bins)[0])
    test_autos_hists.append(np.histogram(test_gs_autos[i], bins=bins)[0])

In [16]:
train_autos_hists = np.load('train_autos_hists.npy')
train_deers_hists = np.load('train_deers_hists.npy')
train_ships_hists = np.load('train_ships_hists.npy')

In [17]:
ship_correct_cntr = 0
auto_correct_cntr = 0
deer_correct_cntr = 0

for i in range(n_samples_test):
    if predict_class(test_ships_hists[i], train_autos_hists, train_ships_hists, train_deers_hists) == 1:
        ship_correct_cntr += 1
    if predict_class(test_deers_hists[i], train_autos_hists, train_ships_hists, train_deers_hists) == 2:
        deer_correct_cntr += 1
    if predict_class(test_autos_hists[i], train_autos_hists, train_ships_hists, train_deers_hists) == 0:
        auto_correct_cntr += 1

In [18]:
ship_acc = ship_correct_cntr / n_samples_test
auto_acc = auto_correct_cntr / n_samples_test
deer_acc = deer_correct_cntr / n_samples_test
overall_acc = (ship_correct_cntr + auto_correct_cntr + deer_correct_cntr) / (n_samples_test * 3)

In [19]:
print('Auto acc: ', '{:.2}'.format(auto_acc))
print('Ship acc: ', '{:.2}'.format(ship_acc))
print('Deer acc: ', '{:.2}'.format(deer_acc))
print('\nOverall acc: ', '{:.2}'.format(overall_acc))

Auto acc:  0.4
Ship acc:  0.2
Deer acc:  0.7

Overall acc:  0.43
